# Converte CSV para Padrão IOB

## Load csv with text and IOB

In [20]:
#!virtualenv --python=python3.8 venv

#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#!pip install seqeval
#!pip install scikit-learn==0.24
#!pip install sklearn==0.0
#!pip install sklearn-crfsuite==0.3.6
#!pip install python-crfsuite==0.9.7
#!pip install tqdm
#!pip install pytorch-crf==0.7.2
#!pip install torch==1.8.1
#!pip install torch==1.11.0
#!pip install torch-summary
# a sessão do colab não possui instalado por padrão



[nltk_data] Downloading package punkt to
[nltk_data]     /home/lucelia_vieira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
import pandas as pd
import nltk
import scipy.stats
import sklearn
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set(color_codes=True)
#sns.set(font_scale=1)
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
nltk.download('punkt')


## Extraindo IOBs do conjunto de dados

In [52]:
#data = pd.read_csv('/home/lucelia_vieira/Experimentos/ner/CSVs/IOBabertura.csv')
data = pd.read_csv('/home/lucelia_vieira/Experimentos/ner/CSVs/DODFCorpus_contratos_licitacoes_v0.csv', dtype=str)
#data = pd.read_csv('/home/lucelia_vieira/Experimentos/ner/CSVs/DODFCorpus_contratos_licitacoes_v0.csv')

In [36]:
data

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto
0,3_4.7.2019,REL_EXTRATO_CONTRATO,R2,anotador_44068453,orgao_contratante,105,anotador_44068453,310795,49,SECRETARIA DE ESTADO DE SAUDE DO DISTRITO FEDERAL
1,3_4.7.2019,REL_EXTRATO_CONTRATO,R2,anotador_44068453,processo_gdf,114,anotador_44068453,312099,22,00060-00144591/2019-21
2,3_4.7.2019,REL_EXTRATO_CONTRATO,R2,anotador_44068453,entidade_contratada,123,anotador_44068453,310857,34,SAMEH -\nSOLUCOES HOSPITALARES LTDA
3,3_4.7.2019,REL_EXTRATO_CONTRATO,R2,anotador_44068453,cnpj_entidade_contratada,124,anotador_44068453,310913,18,25.031.668/0001-27
4,3_4.7.2019,REL_EXTRATO_CONTRATO,R2,anotador_44068453,data_assinatura_contrato,125,anotador_44068453,312143,10,25/06/2019
...,...,...,...,...,...,...,...,...,...,...
158587,634,REL_EXTRATO_CONTRATO,R176,anotador_84982792,unidade_orcamentaria,651,anotador_84982792,285502,6,09.108
158588,634,REL_EXTRATO_CONTRATO,R176,anotador_84982792,programa_trabalho,652,anotador_84982792,285514,21,15.451.6209.1968.0058
158589,634,REL_EXTRATO_CONTRATO,R176,anotador_84982792,nota_empenho,655,anotador_84982792,285582,11,2021NE00476
158590,634,REL_EXTRATO_CONTRATO,R176,anotador_84982792,vigencia_contrato,649,anotador_84982792,285677,243,O prazo de vigência contratual será de 12 (doz...


In [53]:
#lista os labels do dataset
labels = data["tipo_ent"].unique()
labels.sort()
labels

array(['AVISO_ANUL_REV_LICITACAO', 'AVISO_LICITACAO',
       'AVISO_SUSPENSAO_LICITACAO', 'EXTRATO_ADITAMENTO_CONTRATUAL',
       'EXTRATO_CONTRATO', 'EXTRATO_CONVENIO', 'cnpj_entidade_contratada',
       'cnpj_entidade_convenente', 'cnpj_orgao_concedente',
       'cnpj_orgao_contratante', 'codigo_licitacao_sistema_compras',
       'data_abertura_licitacao', 'data_assinatura_contrato',
       'data_assinatura_convenio', 'data_escrito', 'decisao_tcdf',
       'entidade_contratada', 'entidade_convenente', 'fonte_recurso',
       'identificacao_ocorrencia', 'modalidade_licitacao',
       'natureza_despesa', 'nome_responsavel', 'nota_empenho',
       'numero_contrato', 'numero_convenio', 'numero_licitacao',
       'numero_termo_aditivo', 'objeto_aditamento_contratual',
       'objeto_contrato', 'objeto_convenio', 'objeto_licitacao',
       'orgao_concedente', 'orgao_contratante', 'orgao_licitante',
       'processo_gdf', 'programa_trabalho', 'sistema_compras',
       'tipo_objeto', 'unidad

In [54]:
#lista os tipos de atos 
tipo = data["tipo_rel"].unique()
tipo

array(['REL_EXTRATO_CONTRATO', 'REL_ADITAMENTO_CONTRATO',
       'REL_AVISO_LICITACAO', 'REL_ANUL_REVOG_LICITACAO',
       'REL_SUSPENSAO_LICITACAO', 'REL_EXTRATO_CONVENIO'], dtype=object)

In [161]:
tipo_ato = data[data["tipo_rel"]=="REL_EXTRATO_CONVENIO"]

In [162]:
tipo_ato

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
3380,2_19.6.2019,REL_EXTRATO_CONVENIO,R19,anotador_27707309,orgao_concedente,116,anotador_27707309,475498,39,Secretaria de Estado de Saude (SES-\nDF),2_19.6.2019-R19
3381,2_19.6.2019,REL_EXTRATO_CONVENIO,R19,anotador_27707309,orgao_concedente,117,anotador_27707309,475541,50,Fundacao de Ensino e Pesquisa em Ciencias da S...,2_19.6.2019-R19
3382,2_19.6.2019,REL_EXTRATO_CONVENIO,R19,anotador_27707309,objeto_convenio,118,anotador_27707309,475681,416,concessao\nde area para realizacao de estagio ...,2_19.6.2019-R19
3383,2_19.6.2019,REL_EXTRATO_CONVENIO,R19,anotador_27707309,data_assinatura_convenio,119,anotador_27707309,476634,10,29/05/2019,2_19.6.2019-R19
3384,2_19.6.2019,REL_EXTRATO_CONVENIO,R19,anotador_27707309,processo_gdf,120,anotador_27707309,476656,22,00064-00004190/2018-45,2_19.6.2019-R19
...,...,...,...,...,...,...,...,...,...,...,...
154329,635,REL_EXTRATO_CONVENIO,R77,anotador_27707309,vigencia_convenio,1471,anotador_27707309,126613,26,de 27/10/2021 a 26/10/2026,635-R77
154330,635,REL_EXTRATO_CONVENIO,R77,anotador_27707309,valor_convenio,6,anotador_27707309,126651,4,"0,00",635-R77
154331,635,REL_EXTRATO_CONVENIO,R77,anotador_27707309,data_assinatura_convenio,7,anotador_27707309,126691,10,27/10/2021,635-R77
154332,635,REL_EXTRATO_CONVENIO,R77,anotador_27707309,data_assinatura_convenio,1473,anotador_27707309,126127,10,27/10/2021,635-R77


In [6]:
#extrai do csv a coluna texto e tags IOB
#DEFAULT_TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize
#x = []
#y = []
#for row in range(len(data)):
#    if pd.notna(data['Texto'][row]) and pd.notna(data['IOB'][row]):
#        x.append(DEFAULT_TOKENIZER(data['Texto'][row]))
#        y.append(data['IOB'][row].split())
#len(x), len(y)

(1243, 1243)

In [28]:
#z = ''
#for row in range(len(data)) :
#    if pd.notna(data['Texto'][row]) and pd.notna(data['IOB'][row]):
#        # concatenating the columns
#        z = data['Texto'].map(str) + ' ' + data['IOB'].map(str)
#         print(z)
#with open('/home/lucelia_vieira/Experimentos/ner/CSVs/teste_iob.txt', 'wt') as fileout:
#    fileout.writelines(z)

0       AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
1       AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
2       AVISO DE ABERTURA DE LICITACAO PREGAO No 001/2...
3       AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
4       AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
                              ...                        
1238    AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
1239    AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
1240    AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...
1241    AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
1242    AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
Length: 1243, dtype: object
0       AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
1       AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
2       AVISO DE ABERTURA DE LICITACAO PREGAO No 001/2...
3       AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
4       AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
                              ...           

In [74]:
#data['id_ato'] = data['id_dodf'] + '-' + data['id_rel']
#data

In [163]:
tipo_ato['id_ato'] = tipo_ato['id_dodf'] + '-' + tipo_ato['id_rel']
#dfmi.loc[:, ('one', 'second')]

/tmp/ipykernel_84560/2216568828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tipo_ato['id_ato'] = tipo_ato['id_dodf'] + '-' + tipo_ato['id_rel']


In [165]:
tipo_ato

,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
3380,2_19.6.2019,REL_EXTRATO_CONVENIO,R19,anotador_27707309,orgao_concedente,116,anotador_27707309,475498,39,Secretaria de Estado de Saude (SES-\nDF),2_19.6.2019-R19
3381,2_19.6.2019,REL_EXTRATO_CONVENIO,R19,anotador_27707309,orgao_concedente,117,anotador_27707309,475541,50,Fundacao de Ensino e Pesquisa em Ciencias da S...,2_19.6.2019-R19
3382,2_19.6.2019,REL_EXTRATO_CONVENIO,R19,anotador_27707309,objeto_convenio,118,anotador_27707309,475681,416,concessao\nde area para realizacao de estagio ...,2_19.6.2019-R19
3383,2_19.6.2019,REL_EXTRATO_CONVENIO,R19,anotador_27707309,data_assinatura_convenio,119,anotador_27707309,476634,10,29/05/2019,2_19.6.2019-R19
3384,2_19.6.2019,REL_EXTRATO_CONVENIO,R19,anotador_27707309,processo_gdf,120,anotador_27707309,476656,22,00064-00004190/2018-45,2_19.6.2019-R19
...,...,...,...,...,...,...,...,...,...,...,...
154329,635,REL_EXTRATO_CONVENIO,R77,anotador_27707309,vigencia_convenio,1471,anotador_27707309,126613,26,de 27/10/2021 a 26/10/2026,635-R77
154330,635,REL_EXTRATO_CONVENIO,R77,anotador_27707309,valor_convenio,6,anotador_27707309,126651,4,"0,00",635-R77
154331,635,REL_EXTRATO_CONVENIO,R77,anotador_27707309,data_assinatura_convenio,7,anotador_27707309,126691,10,27/10/2021,635-R77
154332,635,REL_EXTRATO_CONVENIO,R77,anotador_27707309,data_assinatura_convenio,1473,anotador_27707309,126127,10,27/10/2021,635-R77


In [51]:
#df2 = df.loc[:, ['A']]
#df2['A'] /= 2    

#df2 = tipo_ato.loc[:,['id_dodf'] + '-' + tipo_ato['id_rel']]

#tipo_ato['id_dodf'] + '-' + tipo_ato['id_rel']
#tipo_ato['id_ato'] = df2
#df2

TypeError: can only concatenate list (not "str") to list

In [166]:
from iob_transformer import iob_transformer
iob = iob_transformer('id_ato', 'texto','tipo_ent', keep_punctuation=True, return_df=False)

In [167]:
atos, labels = iob.transform(tipo_ato)

In [168]:
x=atos
y=labels

In [155]:
y[1]

['O',
 'O',
 'O',
 'B-modalidade_licitacao',
 'O',
 'B-numero_licitacao',
 'O',
 'B-orgao_licitante',
 'I-orgao_licitante',
 'I-orgao_licitante',
 'I-orgao_licitante',
 'I-orgao_licitante',
 'I-orgao_licitante',
 'I-orgao_licitante',
 'I-orgao_licitante',
 'I-orgao_licitante',
 'I-orgao_licitante',
 'I-orgao_licitante',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-modalidade_licitacao',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-processo_gdf',
 'O',
 'O',
 'O',
 'B-modalidade_licitacao',
 'O',
 'B-numero_licitacao',
 'O',
 'O',
 'B-objeto_licitacao',
 'I-objeto_licitacao',
 'I-objeto_licitacao',
 'I-objeto_licitacao',
 'I-objeto_licit

## Create dictionary feature for each word in each sequence in x

In [169]:
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x)):
    x[i] = get_features(x[i])

## Separate train and test splits (in order)

In [170]:
import math
split_idx = math.floor(0.8*len(x))
x_train = x[0:split_idx]
y_train = y[0:split_idx]
x_test = x[split_idx:]
y_test = y[split_idx:]
len(x_train),  len(x_test), len(x)

(14, 4, 18)

## Model CRF Trainning

In [171]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=10,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False,
    all_possible_states=True
)

#crf.fit(x_train, y_train)

try:
    #crf.fit(x_train[:200], y_train[:200])
    crf.fit(x_train, y_train)
except AttributeError:
    pass


## Evaluation

In [172]:
classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.0

In [173]:
print(metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
    y_test, y_pred, labels=classes, digits=3))



                            precision    recall  f1-score   support

         B-numero_convenio      0.000     0.000     0.000         1
        B-orgao_concedente      0.000     0.000     0.000         0
        I-orgao_concedente      0.000     0.000     0.000         0
     B-entidade_convenente      0.000     0.000     0.000         4
     I-entidade_convenente      0.000     0.000     0.000        13
         B-objeto_convenio      0.000     0.000     0.000         4
         I-objeto_convenio      0.000     0.000     0.000       112
       B-vigencia_convenio      0.000     0.000     0.000         4
       I-vigencia_convenio      0.000     0.000     0.000        17
B-data_assinatura_convenio      0.000     0.000     0.000         7
            B-processo_gdf      0.000     0.000     0.000         3
         B-objeto_contrato      0.000     0.000     0.000         0
         I-objeto_contrato      0.000     0.000     0.000         0
         I-numero_convenio      0.000     0.000

/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-numero_convenio', 'B-orgao_concedente', 'I-orgao_concedente', 'B-entidade_convenente', 'I-entidade_convenente', 'B-objeto_convenio', 'I-objeto_convenio', 'B-vigencia_convenio', 'I-vigencia_convenio', 'B-data_assinatura_convenio', 'B-processo_gdf', 'B-objeto_contrato', 'I-objeto_contrato', 'I-numero_convenio', 'B-cnpj_entidade_convenente', 'B-vigencia_contrato', 'I-vigencia_contrato', 'B-valor_convenio', 'B-unidade_orcamentaria', 'B-programa_trabalho', 'B-natureza_despesa', 'B-fonte_recurso', 'B-cnpj_orgao_concedente', 'B-data_assinatura_contrato', 'I-data_assinatura_convenio', 'I-unidade_orcamentaria', 'B-nota_empenho'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-pac

In [14]:
#from sklearn.metrics import confusion_matrix
#confusion_matrix(y_test, y_pred, labels=classes)

## Hyperparameter Optimization

In [ ]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True,
    all_possible_states=True
)
params_space = {
    'c1': scipy.stats.expon(scale=15.0),
    'c2': scipy.stats.expon(scale=1.0),
}


# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=classes)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(x_train, y_train)

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

## Check best estimator on our test data

In [ ]:
sorted_classes = sorted(
    classes,
    key=lambda name: (name[1:], name[0])
)

classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(x_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_classes, digits=3
))

## Let’s check what classifier learned

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

## Saving the best model

In [ ]:
import joblib

joblib.dump(crf, 'crf_model.pkl')

In [ ]:
model = joblib.load('crf_model.pkl')

In [ ]:
model

In [ ]:
model.classes_